Данная тетрадка посвящена Zero-Soht инференсу моделей Gemma2:2b, Gemma2:9b, Mistral-Nemo(12b) с использование библиотеки Unsloth. 

In [1]:
import gc
import re

from tqdm import tqdm
from unsloth import FastLanguageModel
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import torch
import pandas as pd

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/opt/miniconda3/envs/project_news_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
dataset_path = "{your_dataset_path}"
df = pd.read_csv(dataset_path)
df.drop(columns = ['Unnamed: 0'], inplace = True)

print(f"Количество рядов в нашем датасете: {df.shape}")

def remove_garbage(txt):
    new_line_pattern = r"\n\d*"
    brake_pattern = r"<br>"

    txt = re.sub(new_line_pattern, '', txt)
    txt = re.sub(brake_pattern, '', txt)

    return txt

needed_cols = list(df.columns)

for col in needed_cols:
    df[col] = df[col].apply(remove_garbage)


train_df, test_df =  train_test_split(df, test_size = 0.065, random_state = 1337)

train_df = train_df.reset_index().drop(columns = ['index'])
test_df = test_df.reset_index().drop(columns = ['index'])



print(f"Количество примеров в датасете, содержащего gold-examples{test_df.shape}")

train_df.to_csv('{your_dataset_path}')
test_df.to_csv('{your_dataset_path}')

Количество рядов в нашем датасете: (5197, 4)
Количество примеров в датасете, содержащего gold-examples(338, 4)


In [3]:
model_instructions = """Ты - профессиональный журналист с многолетним опытом.
Твоя задача - сгенерировать заголовок новостной статьи, который в максимальной степени отражал бы содержание новости.
Не поясняй свой ответ. Твой финальный ответ должен включать в себя только заголовок и ничего более.
"""

test_df['prompt'] = model_instructions

In [4]:
alpaca_prompt = """Ниже представлены следующие аспекты:
1. Пользовательский запрос.
2. Исходные данные - контекст
3. Ответ - твой ответ.
Сгенерируй ответ, который в полной мере выполняет пользовательский запрос.  
### Пользовательский запрос:
{}

### Исходные данные:
{}

### Ответ:
{}"""

In [5]:
def model_headline_generation(model, tokenizer, input:str)->str:
    '''Функция для генерации новостных заголовок при помощи модели Gemma2:2b-4bit
    
    Args:
        input (str): Входные данные - в нашем случае - текст новостной статьи
    
    Returns:
        str: Сгенерированный новостной заголовок
    '''
    model_instructions = """Ты - профессиональный журналист с многолетним опытом.
    Твоя задача - сгенерировать заголовок новостной статьи, который в максимальной степени отражал бы содержание новости.
    Не поясняй свой ответ. Твой финальный ответ должен включать в себя только заголовок и ничего более.
    """

    inputs = tokenizer(
        [
            alpaca_prompt.format(
                model_instructions,
                input,
                '',
            )
        ],return_tensors = 'pt').to('cuda')
    
    outputs = model.generate(**inputs, max_new_tokens = 200, use_cache = True)

    result = tokenizer.batch_decode(outputs)

    answer_pattern = r"Ответ:[^<eos>]*"

    return re.findall(answer_pattern, result[0])[0]

In [6]:
def clear_cache(model, tokenizer) -> None:
    '''Функция удаляет модель из GPU'''
    with torch.no_grad():
        model.cpu()
    del model
    del tokenizer
    
    gc.collect()
    
    torch.cuda.empty_cache()

Генерация новостных заголовков методом Zero-Shot

In [ ]:
tqdm.pandas()

models_list = ['unsloth/gemma-2-2b',
               'unsloth/gemma-2-9b-bnb-4bit',
               'unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit']

for model in models_list:

    model_name = model

    print(f"Происходит инициализация модели {model}")

    model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model,
    max_seq_length = 1048,
    dtype = None,
    load_in_4bit = True
    )
    
    FastLanguageModel.for_inference(model)

    print("Выполняется генерация...")

    test_df[f'{model_name}_headline'] = test_df['Текст новости'].progress_apply(lambda x: model_headline_generation(model = model,
                                                                                                               tokenizer = tokenizer,
                                                                                                               input = x))
    clear_cache(model = model,
                tokenizer = tokenizer)

test_df.to_csv('{your_dataset_path}')
